In [1]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

In [2]:
img = cv2.imread('capturedIMG.png')

In [3]:
yolo_net = cv2.dnn.readNet("models/yolov4-tiny-detector_best_pisciculturedb.weights", "models/yolov4-tiny_testing_3chan.cfg")

caffe_net = cv2.dnn.readNetFromCaffe("models/MobileNetSSD_deploy.prototxt", "models/MobileNetSSD_deploy.caffemodel")

In [4]:
yolo_layer_names = yolo_net.getLayerNames()
yolo_output_layers = [yolo_layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]

# caffe_layer_names = yolo_net.getLayerNames()
# caffe_output_layers = [caffe_layer_names[i[0] - 1] for i in caffe_net.getUnconnectedOutLayers()]

In [5]:
height, width, channels = img.shape # Yolo
(H, W) = img.shape[:2]              # Caffe

In [6]:
yolo_blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
caffe_blob = cv2.dnn.blobFromImage(img, 0.007843, (W, H), 127.5)

In [7]:
print(f"yolo_blob.shape: {yolo_blob.shape}")
print(f"caffe_blob.shape: {caffe_blob.shape}")

yolo_blob.shape: (1, 3, 416, 416)
caffe_blob.shape: (1, 3, 480, 640)


In [8]:
yolo_net.setInput(yolo_blob)
yolo_outs = yolo_net.forward(yolo_output_layers)

caffe_net.setInput(caffe_blob)
caffe_detections = caffe_net.forward()

## Yolo Inference

In [9]:
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in yolo_outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.0:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
num_peces = 0

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        num_peces += 1
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv2.rectangle(color_img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(color_img, label, (x, y + 30), font, 2, color, 2)

In [10]:
boxes[0]

[317, 232, 89, 75]

## Caffe Inference

In [11]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

# loop over the detections
for i in np.arange(0, caffe_detections.shape[2]):
    # extract the confidence (i.e., probability) associated
    # with the prediction
    confidence = caffe_detections[0, 0, i, 2]

    # filter out weak detections by requiring a minimum
    # confidence
    if confidence > 0.5:
        # extract the index of the class label from the
        # detections list
        idx = int(caffe_detections[0, 0, i, 1])

        # if the class label is not a person, ignore it
        if CLASSES[idx] != "person":
            continue

        # compute the (x, y)-coordinates of the bounding box
        # for the object
        box = caffe_detections[0, 0, i, 3:7] * np.array([W, H, W, H])
        (startX, startY, endX, endY) = box.astype("int")

In [12]:
box

array([300.26351929, 153.65833282, 583.85047913, 446.86712265])